In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_customer
COMMENT 'Customers bronze table'
TBLPROPERTIES('quality'='bronze'
               ,'delta.autoOptimize.optimizeWrite' = 'true',
               'delta.autoOptimize.autoCompact' = 'true'
)
AS
SELECT *,_metadata.file_path as input_filepath,
      
       current_timestamp() as ingestion_timestamp
FROM cloud_files(
    '/Volumes/dlt_pipeline_catalog/landing_schema/project_dlt_volume/customers/',
    'json',
    map(
       'cloudFiles.inferColumnTypes','true',
       'cloudFiles.schemaEvolutionMode','addNewColumns'
       )     
    )

In [0]:
set spark.databricks.delta.schema.autoMerge.enable=true

In [0]:
CREATE OR REFRESH STREAMING  TABLE customers_quarantine
AS
SELECT *
FROM STREAM(LIVE.bronze_customer)
WHERE customer_id IS NULL;

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_customer_clean(
  CONSTRAINT customer_notnull expect(customer_id is not null) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_name  EXPECT(customer_name is not null) ON VIOLATION DROP ROW,
  CONSTRAINT valid_email EXPECT(email is not null),
  CONSTRAINT valid_telephone EXPECT(length(telephone)>=10),
  CONSTRAINT valid_date EXPECT(length(date_of_birth) >='1920-01-01')
)
TBLPROPERTIES('quality'='silver',
              'delta.autoOptimize.optimizeWrite' = 'true',
              'delta.autoOptimize.autoCompact' = 'true'
              
              )
AS
SELECT customer_id
      ,customer_name
      ,cast(date_of_birth as date) as date_of_birth
      ,email
      ,telephone
      ,cast(current_date as date) as created_date

FROM STREAM(LIVE.bronze_customer) where customer_id is not null

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_customer
comment'Silver table for customer'
TBLPROPERTIES('quality'='silver')

In [0]:
APPLY CHANGES INTO silver_customer
FROM STREAM(LIVE.silver_customer_clean)
KEYS(customer_id)
SEQUENCE BY created_date
STORED AS scd TYPE 1